In [1]:
from augment import *
import glob
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from GAN import *                  
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #

In [2]:
def dstnc(dt2,dt1): #فاصله و واریانس بردار فاصله ی هر کدام از ردیف های دیتاست(آگمنت)1 از (کل)2
    dist,vrnc,i,h,w=[],[],0,np.shape(dt1)[0],np.shape(dt1)[1]
    end=min(len(dt2[0]),len(dt1[0]))
    for i in range (h):
        ds1 = dt2-dt1[i,:end] #.reshape(-1,1) #distance of every training instance
        dist_array=(ds1*10)**3  #معیار فاصله با حساسیت به توان 3 (حفظ علامت و تاثیر بیشتر تفاوت زیاد در نقاط)
        vr=np.var(ds1)
        ds1=min(np.sum(np.absolute(dist_array),axis=1))
        #print('dist2 = ', dist2)
        vrnc.append(vr)
        dist.append(ds1)
    variance=np.reshape(vrnc,(-1,1))
    distnc=np.reshape(dist,(-1,1))
    return(distnc,variance)

cls_num=12
ecg=np.array([])
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=EOGHorizontalSignal(i)         #فراخوانی داده های اصلی
    ecg=np.append(ecg,vars()['ecg'+str(i)])
wdth=int(len(ecg1[0]))
ecg=np.reshape(ecg,(int(len(ecg)/wdth),wdth))  #base data windows
print('\nfinal_train_shap=',np.shape(ecg))
os.chdir(cwd)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (362, 1251)
first_test_shape= (362, 1251)
classes_quantity= 12
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
Count_labels= [30 30 30 30 30 31 31 30 30 30 30 30]
max(train_feature_Altitude)= 446.32
min(train_feature_Altitude)= -1110.8
first_train_sample=
 [   1.         0.47769    0.60965 ... -110.76    -111.03    -110.9    ]

final_train_shap= (362, 481)


In [3]:
base_wn=np.array([])
windws=np.array([])

for cls in range (1,cls_num+1):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])
    base_wn=np.append(base_wn,vars()['wndws'+str(cls)])
base_wn=np.reshape(base_wn,(int(len(base_wn)/wdth),wdth))  #base data windows
print("shape wndws1=", np.shape(wndws1))
print("shape base_wn=", np.shape(base_wn))

furir_windws0=np.fft.fft(np.array(base_wn[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([base_wn[:,-1]])),axis=1)   
print("shape furir_windws=", np.shape(furir_windws))

smpl_rte=len(base_wn[0])-1                 #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
print("smpl_rte=",smpl_rte)
mx_wndws=np.max(np.bincount(np.int16(base_wn[:,-1])))
print("mx_wndws=",mx_wndws)

shape wndws1= (30, 481)
shape base_wn= (362, 481)
shape furir_windws0= (362, 480)
shape furir_windws= (362, 481)
smpl_rte= 480
mx_wndws= 31


In [4]:
path=('{}\AugEvlH'.format(cwd)) #create folder 
if not os.path.isdir(path):
    os.mkdir(path)

def addqlty(base_wn_dt,data,mthd,i):               
    cls=int(i)
    c,alpha,PL=0,1,1
    #from glob import glob
    cwd = os.getcwd() #
    adrs=cwd+'/Aug/aug'+str(i)+str(mthd)+'.csv'
    #alldata = np.array([])                    # List for storing all the data
    train_labels=np.array(base_wn_dt[:,-1])
    tr=np.array(base_wn_dt[:,:-1])
    indx_pstv=np.where(base_wn_dt[:,-1] == i)
    P=np.array(base_wn_dt[indx_pstv])
    indx_ngtv=np.where(base_wn_dt[:,-1] != i)
    N=np.array(base_wn_dt[indx_ngtv])

    #data = np.array(data)
    aug=np.array(data)
    pnts=int(np.shape(data)[1])
    
    #aug=data[data[:,-2].argsort()]        # Positive data Sorted Base Distance
    trl=np.array(train_labels)
    for j in range (len(trl)):
        if train_labels[j]==i :           # !!!!! == (Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    #mn_ds_pstv=np.reshape(aug[:,-2],(-1,1)) #min dis to positive class (submited in -2 column)
    print('shape and aug[0]=',np.shape(aug[0]),aug[0])
    print('shape P and P[0]=',np.shape(P),P[0])    
    qlty1,qlty2=mn_ds_pstv,vr_P=dstnc(P[:,1:-1],aug[:,1:-1])  #min distance to negative class
    new1=np.append(aug,qlty1,axis=1)
    new2=np.append(new1,qlty2,axis=1)
    mn_ds_ngtv,vr_N=dstnc(N[:,1:-1],aug[:,1:-1])              #min distance to negative class
    qlty3=mn_ds_pstv/mn_ds_ngtv                               #Smaller is Better
    qlty4=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new3=np.append(new2,qlty3,axis=1)
    new4=np.append(new3,qlty4,axis=1)
    for j in range (len(trl)):
        if train_labels[j]!=i :                          # !!!!! == (!= Inverse Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    qlty5=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new5=np.append(new4,qlty5,axis=1)
    print('class=',i,'shape(new5)=',np.shape(new5),end='\t')
    print('mx=',np.max(new5[:,1:-7]),end='\t')
    print('mn=',np.min(new5[:,1:-7]))
    pd.DataFrame(new5[:,:]).to_csv('{}/AugEvlH/AugEvl{}{}.csv'.format(cwd,cls,mthd))
    print()

In [5]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+'00']=np.empty((0,smpl_rte+1), float) #Scale    
    vars()['ecg_aug'+str(i)+'10']=np.empty((0,smpl_rte+1), float) #Scale
    vars()['ecg_aug'+str(i)+'20']=np.empty((0,smpl_rte+1), float) #Mag Warp
    vars()['ecg_aug'+str(i)+'30']=np.empty((0,smpl_rte+1), float) #TimWarp
    vars()['ecg_aug'+str(i)+'40']=np.empty((0,smpl_rte+1), float) #GDO
    vars()['ecg_aug'+str(i)+'50']=np.empty((0,smpl_rte+1), float) #frequence
    vars()['ecg_aug'+str(i)+'60']=np.empty((0,smpl_rte+1), float) #GAN
    vars()['ecg_aug'+str(i)+'61']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'62']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'70']=np.empty((0,smpl_rte+1), float) #LSTM
    vars()['ecg_aug'+str(i)+'71']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'72']=np.empty((0,smpl_rte+1), float)

In [6]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vlum_sig=10*mx_wndws-len(vars()['wndws'+str(i)])
    
    vars()['ecg_aug'+str(i)+'10']=np.append(vars()['ecg_aug'+str(i)+'10'],sclRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)#0.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'10'],10,i)
    print(str(i)+'10 =',len(vars()['ecg_aug'+str(i)+'10']))
    
    vars()['ecg_aug'+str(i)+'20']=np.append(vars()['ecg_aug'+str(i)+'20'],MgWrpRaw(ecg,i,int(vlum_sig),mag=.05),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'20'],20,i)
    print(str(i)+'20 =',len(vars()['ecg_aug'+str(i)+'20']))
    
    vars()['ecg_aug'+str(i)+'30']=np.append(vars()['ecg_aug'+str(i)+'30'],TimWrpRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)#.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'30'],30,i)
    print(str(i)+'30 =',len(vars()['ecg_aug'+str(i)+'30']))
    
    vars()['ecg_aug'+str(i)+'40']=np.append(vars()['ecg_aug'+str(i)+'40'],GDORaw(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'40'],40,i)
    print(str(i)+'40 =',len(vars()['ecg_aug'+str(i)+'40']))
    
    vars()['ecg_aug'+str(i)+'50']=np.append(vars()['ecg_aug'+str(i)+'50'],frqncRaw(furir_windws,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'50'],50,i)
    print(str(i)+'50 =',len(vars()['ecg_aug'+str(i)+'50']))

shape and aug[0]= (481,) [-9.17500000e+01  2.04531250e+01 -1.41875000e+01  6.25000000e+00
 -8.07812500e+00  3.29492188e+00 -6.07812500e+00  1.53320312e+00
 -5.30859375e+00  3.02978516e-01 -5.04296875e+00 -1.89697266e-01
 -4.55468750e+00 -6.71875000e-01 -3.68750000e+00  1.52539062e+00
  1.14746094e+00  6.06640625e+00  4.54296875e+00  7.76953125e+00
  4.89843750e+00  7.66015625e+00  4.65625000e+00  5.71875000e+00
  3.81054688e+00  5.29296875e+00  3.23242188e+00  3.84765625e+00
  1.05273438e+00  2.08203125e+00 -7.13867188e-01  8.61816406e-01
  3.92822266e-01  6.99707031e-01 -5.48339844e-01  7.57812500e-01
 -6.97265625e-01  1.56494141e-01 -1.43164062e+00 -1.76562500e+00
 -2.50976562e+00 -1.32421875e+00 -2.05859375e+00 -1.24121094e+00
 -1.66406250e+00 -9.45800781e-01 -1.45605469e+00  2.37915039e-01
 -5.09277344e-01  1.29980469e+00  1.91210938e+00  3.13671875e+00
  2.34570312e+00  2.40820312e+00  1.05273438e+00  1.52050781e+00
 -1.07650757e-02 -1.66406250e+00 -2.54687500e+00 -2.77734375e+00


class= 1 shape(new5)= (283, 486)	mx= 338.75	mn= -1529.0

110 = 283
shape and aug[0]= (481,) [-4.00113362e+02  4.14639941e+00 -1.20147517e+02 -4.87229178e+01
 -1.02535125e+02 -6.41366525e+01 -9.88796858e+01 -7.47336562e+01
 -9.99220908e+01 -8.15550620e+01 -1.02697064e+02 -8.85041553e+01
 -1.04055117e+02 -9.12667931e+01 -1.04779742e+02 -9.16465631e+01
 -9.84365591e+01 -8.45225786e+01 -9.03864901e+01 -6.09018374e+01
 -3.78959565e+01 -6.70061801e+00 -1.38251251e+01 -4.28677284e+00
 -7.03114069e+00  5.41305468e+00  3.46425908e+00  2.64509022e+01
  5.56756234e+01  9.05168410e+01  1.05716309e+02  1.16617539e+02
  1.15980386e+02  1.20174836e+02  1.14830949e+02  1.17717197e+02
  1.11108009e+02  1.13573532e+02  1.07782953e+02  1.09646610e+02
  1.04089031e+02  1.06663562e+02  1.01808081e+02  1.03026959e+02
  9.60435165e+01  9.19313999e+01  8.51761958e+01  8.23597982e+01
  7.36613745e+01  7.12045562e+01  6.27874912e+01  6.37151403e+01
  5.77155817e+01  5.70023983e+01  5.17386345e+01  5.16959076e+0

class= 1 shape(new5)= (283, 486)	mx= 269.58601749283747	mn= -1189.0953885109661

120 = 283
shape and aug[0]= (481,) [-3.37058814e+01 -2.09202836e+01 -8.13468575e+00  4.65091208e+00
  6.49887241e+00  2.61469008e+00 -1.26949225e+00 -2.87143701e+00
 -4.04334677e-01  2.06276765e+00  4.06387011e+00  2.26204546e+00
  4.60220807e-01 -1.34160384e+00 -4.23439285e-01  7.79659598e-01
  1.98275848e+00  1.25929257e+00 -1.16856734e-01 -1.49300604e+00
 -1.43101554e+00 -5.38936079e-01  3.53143378e-01  2.31616049e-01
 -6.67110412e-01 -1.56583687e+00 -1.46024924e+00 -5.07432718e-01
  4.45383807e-01  5.49076864e-01 -6.74098212e-03 -5.62558828e-01
 -3.25650839e-01  3.83176408e-01  1.09200365e+00  9.99853145e-01
  6.17379605e-01  2.34906066e-01  7.20192382e-01  1.31957599e+00
  1.83618128e+00  1.34472821e+00  8.53275139e-01  5.94160917e-01
  8.52667397e-01  1.11117388e+00  7.78789410e-01  2.74123712e-02
 -7.23964668e-01 -7.65158890e-01 -7.28640837e-01 -8.91839471e-01
 -1.62898824e+00 -2.36613700e+00 -2.397

class= 1 shape(new5)= (283, 486)	mx= 254.87029826304425	mn= -1159.668721736098

130 = 283
shape and aug[0]= (481,) [-8.46225624e+01  3.12436857e+01 -1.53177749e+00  2.11558657e+01
  6.87382256e+00  1.65045796e+01  4.46492197e+00  1.08364022e+01
  1.94658012e+00  6.37315046e+00  1.34359614e-01  4.08072364e+00
 -9.61272692e-01  3.46980103e+00 -3.87001658e-01  2.78618443e+00
 -8.15484068e-01  1.86640136e+00 -1.20805851e+00  1.05600565e+00
 -1.96157059e+00 -2.87838847e-02 -2.60268998e+00 -1.11505389e+00
 -4.41103855e+00 -3.14110881e+00 -5.25965666e+00 -3.56796465e+00
 -5.12967164e+00 -3.29316991e+00 -4.51731207e+00 -3.16331179e+00
 -4.78626421e+00 -3.50264655e+00 -4.61141911e+00 -3.44662234e+00
 -4.32342750e+00 -3.34108065e+00 -4.95374148e+00 -4.34748288e+00
 -5.64507946e+00 -4.52969665e+00 -5.74880197e+00 -5.12142491e+00
 -6.20204217e+00 -5.71621134e+00 -6.97702486e+00 -5.87208180e+00
 -6.65222830e+00 -6.48179200e+00 -6.04500922e+00 -4.75578460e+00
 -5.56159407e+00 -4.30610339e+00 -4.9818

class= 1 shape(new5)= (283, 486)	mx= 256.85226474057214	mn= -1417.8332085136879

140 = 283


...

class= 12 shape(new5)= (283, 486)	mx= 449.0	mn= -225.375

1210 = 283
shape and aug[0]= (481,) [ 25.33552389  17.41232413  19.68590042  18.71939111  19.39174148
  19.32205442  18.52880116  15.87870152  14.7442923   14.30679973
  14.25439414  12.41827748   9.96920979   6.01122824   4.93435583
   1.4943707    0.9954915   -5.74911217 -10.47402199 -13.98712875
 -15.97072282 -15.54127725 -11.12069765 -10.31852042 -11.17506607
  -9.85338326  -9.13597394 -10.14251106  -6.0503215   -8.13086022
  -9.65526056 -10.23543952  -7.11728384  -5.52134927  -9.5618721
 -10.58475323 -10.79064273  -8.10699316  -9.09186874  -7.92461878
  -8.13285775  -7.76812279  -4.86320097  -1.86525099  -6.29166119
  -5.47921181  -7.142539    -3.89323388  -4.30447502  -2.71705132
  -5.5177957   -3.73196224  -2.40471106  -3.53133403  -3.35191645
  -4.44850442  -3.53197521  -4.25059481  -3.40641638  -3.89659518
  -2.31570902  -1.58573221   1.14782978   0.37253823   2.44029971
   1.50810536   0.88554129  -0.37618855  -1.40546

class= 12 shape(new5)= (283, 486)	mx= 370.6623355907678	mn= -137.44162770516198

1220 = 283
shape and aug[0]= (481,) [ 4.42656538e+01  3.95938159e-01  8.63134641e+00  2.02581688e+00
  4.86653282e+00  2.97198161e+00  3.40345037e+00  6.76676816e-01
 -6.79907985e-01  1.46688620e-01 -8.44890542e-01 -2.97597953e-01
 -1.15008729e+00  8.11888724e-02 -1.50158972e+00  2.24892272e-01
 -5.11326657e-01  6.34162668e-01  5.44593370e-01  4.95069966e-01
  5.41260650e-01  1.41285530e-01 -1.80023097e+00 -2.17705737e+00
 -3.10249750e+00 -1.73756779e+00 -2.52652724e+00 -5.74272338e-01
 -5.87637258e-01 -6.73054837e-02 -3.39031009e-01 -1.17804830e+00
 -2.35272287e+00 -3.41804444e+00 -4.35879611e+00 -4.42759682e+00
 -4.50046273e+00 -3.84662642e+00 -2.84656508e+00 -6.74471351e-01
  1.65618928e+00  2.44776946e+00  3.26829435e+00  3.03988000e+00
  4.91635594e+00  1.40298444e+00  1.13307814e+00 -2.43280579e-01
 -3.41874989e-01 -1.60538768e+00 -2.83258695e+00 -3.36017897e+00
 -3.83665651e+00 -4.30609237e+00 -4.01

class= 12 shape(new5)= (283, 486)	mx= 352.49045716604365	mn= -173.85706191625417

1230 = 283
shape and aug[0]= (481,) [ 4.50381621e+01 -2.80407726e-01  1.00284236e+01  9.78291570e-01
  5.99408981e+00  1.69671739e+00  4.95999783e+00  8.18381728e-01
  4.15233204e-01 -1.68625313e+00  1.15455181e+00 -1.60194238e+00
  1.16895031e-01 -1.35662727e+00  1.41565041e-01 -1.54022429e+00
  4.06231116e-01 -8.18032510e-01  7.46635153e-01  3.79011072e-01
  8.35623715e-01  1.01302744e-01  9.30694530e-01 -3.46280591e-01
 -2.39230218e+00 -2.11458593e+00 -3.24450047e+00 -1.71343170e+00
 -2.77803091e+00 -5.29000041e-01 -8.46205880e-01  4.99886510e-02
 -2.66900701e-01 -4.21858643e-01 -1.70451986e+00 -2.65993548e+00
 -3.73864454e+00 -4.43115237e+00 -4.47430677e+00 -4.61638280e+00
 -3.87692853e+00 -3.70598059e+00 -1.01828438e+00 -1.80269840e-01
  3.15491095e+00  2.06273829e+00  3.67821935e+00  3.02700840e+00
  4.96650666e+00  1.37068932e+00  1.69405336e+00 -5.80429107e-01
  3.83007189e-01 -1.09842126e+00 -1.8

class= 12 shape(new5)= (283, 486)	mx= 373.8148789912698	mn= -180.00706039387774

1240 = 283
shape and aug[0]= (481,) [ 1.78125000e+01 -4.42578125e+00  4.21484375e+00  4.34814453e-01
  4.58984375e+00  1.67480469e+00  3.36718750e+00  1.53222656e+00
  1.86621094e+00 -1.34472656e+00 -1.11230469e+00 -2.61718750e+00
 -1.62597656e+00 -1.98242188e+00 -8.17871094e-01 -6.02539062e-01
 -1.59545898e-01 -4.57031250e-01  1.42700195e-01 -2.67822266e-01
  5.03906250e-01  2.62298584e-02  2.18383789e-01 -6.16699219e-01
 -9.57031250e-01 -1.43457031e+00 -1.81738281e+00 -2.38085938e+00
 -2.87304688e+00 -3.26757812e+00 -3.14062500e+00 -2.16406250e+00
 -1.32226562e+00 -1.84472656e+00 -1.15722656e+00 -4.91943359e-01
  1.35253906e+00  2.35742188e+00  2.80859375e+00  2.93945312e+00
  3.33789062e+00  2.93750000e+00  2.61132812e+00  2.33203125e+00
  1.86621094e+00  3.48632812e-01 -9.13574219e-01 -1.40136719e+00
 -1.90234375e+00 -2.37890625e+00 -3.11328125e+00 -1.81738281e+00
 -1.05859375e+00 -1.10937500e+00 -8.56

class= 12 shape(new5)= (283, 486)	mx= 357.75	mn= -135.5

1250 = 283
